In [ ]:
# Initialize OK
from client.api.notebook import Notebook
ok = Notebook('project2.ok')

# Project 2: Climate Prediction
## Due date: December 11, 5PM.

In [95]:
# Don't change this cell; just run it. 

import numpy as np
from datascience import *

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

from client.api.notebook import Notebook

Comment-out the cell below to make sure that you create a backup of your project on okpy.org.

In [96]:
#_ = ok.submit()

## Predicting Temperatures

In this exercise, we will try to predict the weather in California using the prediction method  discussed in [section 8.1 of the textbook](https://www.inferentialthinking.com/chapters/08/1/Applying_a_Function_to_a_Column).  Much of the code is provided for you; you will be asked to understand and run the code and interpret the results.

The US National Oceanic and Atmospheric Administration (NOAA) operates thousands of climate observation stations (mostly in the US) that collect information about local climate.  Among other things, each station records the highest and lowest observed temperature each day.  These data, called "Quality Controlled Local Climatological Data," are publicly available [here](http://www.ncdc.noaa.gov/orders/qclcd/) and described [here](https://www.ncdc.noaa.gov/data-access/land-based-station-data/land-based-datasets/quality-controlled-local-climatological-data-qclcd).

`temperatures.csv` contains an excerpt of that dataset.  Each row represents a temperature reading in Fahrenheit from one station on one day.  (The temperature is actually the highest temperature observed at that station on that day.)  All the readings are from 2015 and from California stations.

In [97]:
temperatures = Table().read_table("temperatures.csv")
temperatures

Here is a scatter plot:

In [98]:
temperatures.scatter("Date", "Temperature")
_ = plots.xticks(np.arange(0, max(temperatures.column('Date')), 100), rotation=65)

Each entry in the column "Date" is a number in MMDD format, meaning that the last two digits denote the day of the month, and the first 1 or 2 digits denote the month. Thus, December 31st (12/31) is represented as `1231` and January 31st (1/31) is `131`.

**Question 1.1** Why do the data form vertical bands with gaps?

_Hint_: Read the paragraph above about how "Date" is represented.

<!--
BEGIN QUESTION
name: q1_vertical_bands
manual: true
points: 2
-->
<!-- EXPORT TO PDF -->

<hr style="color:Maroon;background-color:Maroon;border:0 none; height: 3px;">
Replace this text with your answer.
<hr style="color:Maroon;background-color:Maroon;border:0 none; height: 3px;">

We will convert each date to the number of days since the start of the year.<br>

**Question 1.2** Implement the `get_day_in_month` function, as described below. The result should be an integer.<br>
_Hint:_ Use the [remainder operator](https://www.inferentialthinking.com/chapters/03/1/Expressions).

<!--
BEGIN QUESTION
name: q1_get_day
manual: false
points: 2
-->

In [99]:
def get_month(date):
    
    """The month in the year for a given date.
    
    >>> get_month(315)
    3
    """
    return int(date / 100) # Divide by 100 and round down to the nearest integer

def get_day_in_month(date):
    
   
    """The day in the month for a given date.
    
    >>> get_day_in_month(315)
    15
    """
    ...

In [ ]:
ok.grade("q1_get_day");

Next, we'll compute the *day of the year* for each temperature reading, which is the number of days from January 1 until the date of the reading. Note that we are not dealing with a leap year (a year with 366 days).

In [103]:
# You don't need to change this cell, but you are strongly encouraged
# to read all of the code and understand it.

days_in_month = make_array(31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31)

# A table with one row for each month.  For each month, we have
# the number of the month (e.g. 3 for March), the number of
# days in that month in 2015 (e.g. 31 for March), and the
# number of days in the year before the first day of that month
# (e.g. 0 for January or 59 for March).
days_into_year =  Table().with_columns(
    "Month", np.arange(12)+1,
    "Days until start of month", np.cumsum(days_in_month) - days_in_month)

# First, compute the month and day-of-month for each temperature.
months = temperatures.apply(get_month, "Date")
day_of_month = temperatures.apply(get_day_in_month, "Date")
with_month_and_day = temperatures.with_columns(
    "Month", months,
    "Day of month", day_of_month
)

# Then, compute how many days have passed since 
# the start of the year to reach each date.
t = with_month_and_day.join('Month', days_into_year)
day_of_year = t.column('Days until start of month') + t.column('Day of month')
with_dates_fixed = t.drop(0, 6, 7).with_column("Day of year", day_of_year)
with_dates_fixed

Do we have values for all days in a year? Let's find out by checking if any days are missing.

**Question 1.3** Set `missing` to an array of all the days of the year (integers from 1 through 365) that do not have any temperature readings.<br>
*Hint:* One possible strategy (but not the only one) is to start with a table of all days in the year, then use either the predicate `are.not_contained_in` ([docs](http://data8.org/datascience/predicates.html)) or the method `exclude` ([docs](http://data8.org/datascience/_autosummary/datascience.tables.Table.exclude.html#datascience.tables.Table.exclude))  to eliminate all of the days of the year that do have a temperature reading. 

<!--
BEGIN QUESTION
name: q1_missing
manual: false
points: 2
-->

In [104]:
#all_days_with_data = with_dates_fixed.group(...).column("Day of year") # get the array of all days for which we have data
missing = ...
missing

In [ ]:
ok.grade("q1_missing");

Using `with_dates_fixed`, we can make a better scatter plot.

In [109]:
with_dates_fixed.scatter("Day of year", "Temperature")

Let's do some prediction.  For any reading on any day, we will predict its value using **all the readings from _the week before and after_ that day**.  A reasonable prediction is that the predicted reading will be **the average of all those readings**.  Let's package our code in a function.

In [110]:
def predict_temperature(day):
    """
    A prediction of the temperature (in Fahrenheit) on a given day at some station.
    Relies on a `with_dates_fixed` table that has the "Day of year" and "Temperature" columns.
    """
    nearby_readings = with_dates_fixed.where("Day of year", are.between_or_equal_to(day - 7, day + 7))
    #print(nearby_readings)
    return np.average(nearby_readings.column("Temperature"))

Let's see how well it works by looking at the values on a random day and seeing what is the predicted temperature on that day. For example, day 223 is August 11 (8/11, hence the `Date` as `811`) and the temperature varied a lot on that day, depending on the location.

In [111]:
with_dates_fixed.where("Day of year", 223)

In [112]:
predict_temperature(223)

**Question 1.4** Suppose you're planning a trip to Yosemite for the day after your INT 5 final, Dec 11, and you'd like to predict the temperature on Dec 11. Use `predict_temperature` to compute a prediction for a temperature reading on that day.

_Hint_: You can figure out what day of the year Dec 11 is by searching `with_dates_fixed` for a similar date or by using the fact that Dec 31st is day 365.
<!--
BEGIN QUESTION
name: q1_dec11
manual: true
points: 3
-->
<!-- EXPORT TO PDF -->

In [113]:
# convert the date for Dec 11 into a "day of year" value
dec_11_day = ...

dec_11_prediction = ...
dec_11_prediction

In [ ]:
ok.grade("q1_dec11");

Below we have computed a predicted temperature for each reading in the table and plotted both.  (It may take a **minute or two** to run the cell.)

In [117]:
with_predictions = with_dates_fixed.with_column(
    "Predicted temperature",
    with_dates_fixed.apply(predict_temperature, "Day of year"))
with_predictions.select("Day of year", "Temperature", "Predicted temperature")\
                .scatter("Day of year")

**Question 1.5** The scatter plot is called a *graph of averages*.  In the [example in the textbook](https://www.inferentialthinking.com/chapters/08/1/Applying_a_Function_to_a_Column), the graph of averages roughly followed a straight line.  Is that true for this one?  Using your knowledge about the weather, explain why or why not.

<!--
BEGIN QUESTION
name: q1_graph_averages
manual: true
points: 3
-->
<!-- EXPORT TO PDF -->

<hr style="color:Maroon;background-color:Maroon;border:0 none; height: 3px;">
Replace this text with your answer.
<hr style="color:Maroon;background-color:Maroon;border:0 none; height: 3px;">

**Question 1.6** According to the [Wikipedia article](https://en.wikipedia.org/wiki/Climate_of_California) on California's climate, "[t]he climate of California varies widely, from hot desert to subarctic."  Suppose we limited our data to weather stations in a smaller area whose climate varied less from place to place (for example, the state of Vermont, or the San Francisco Bay Area). If we made the same graph for that dataset, in what ways would you expect it to look different? In what ways would you expect it to look the same? 

<!--
BEGIN QUESTION
name: q1_similarities
manual: true
points: 3
-->
<!-- EXPORT TO PDF -->

<hr style="color:Maroon;background-color:Maroon;border:0 none; height: 3px;">
Replace this text with your answer.
<hr style="color:Maroon;background-color:Maroon;border:0 none; height: 3px;">

___

## Rainfall and Temperatures

In this exercise we will now look at data from the United States on Rainfall and Temperature. The goal is to find out if there is a correlation between temperature change and rainfall. The visualization below shows how the temperature has risen over the years. Thus, investigating the affect of this temperature change on rainfall is important!

<img src="http://www.climate-lab-book.ac.uk/files/2016/06/spiral_2017_large-1.gif" style="height:500px;" />

#### Merging Datasets
The data for rainfall and temperature are in two different datasets, so we will need to merge them before we do any prediction. Below we begin by reading in the data tables. The dataset has extra columns that we don't need, and also column labels that have a misleading space, so we select only the data we need and relabel the columns.

In [118]:
usa_rainfall = Table.read_table("pr_1901_2016_USA.csv")\
    .relabel(" Year", "Year")\
    .relabel(" Statistics", "Statistics")\
    .select("Rainfall - (MM)","Year","Statistics")
usa_rainfall

In [119]:
usa_temperatures = Table.read_table("tas_1901_2016_USA.csv")\
    .relabel(" Year", "Year")\
    .relabel(" Statistics", "Statistics")\
    .select("Temperature - (Celsius)","Year","Statistics")
usa_temperatures

In [120]:
usa_rain_and_temp = usa_rainfall.with_column("Temperature - (Celsius)", 
                                             usa_temperatures.column("Temperature - (Celsius)"))
usa_rain_and_temp

Since we know the data is arranged the exact same way between both tables, we can just join by copying over the Temperature column. It is much easier that way.

In [121]:
usa_rain_and_temp = usa_rainfall.with_column("Temperature - (Celsius)", 
                                             usa_temperatures.column("Temperature - (Celsius)"))
usa_rain_and_temp

**Question 2.1** Create a scatterplot with the Temperature on the x-axis and Rainfall on the y-axis. 

In [122]:
# Enter code here to visualize Temperature vs. Rainfall
...

Comment on your observations about the plot. Do you think there is a relationship between Temperature and Rainfall? If so, what kind of relationship do you see?

<!--
BEGIN QUESTION
name: q2_relationship
manual: true
points: 3
-->
<!-- EXPORT TO PDF -->

<hr style="color:Maroon;background-color:Maroon;border:0 none; height: 3px;">
Replace this text with your answer.
<hr style="color:Maroon;background-color:Maroon;border:0 none; height: 3px;">

**Question 2.2** Using the standard units function from the book in [chapter 15.1](https://www.inferentialthinking.com/chapters/15/1/Correlation.html), compute the rainfall, and temperate in standard units.

In [123]:
def standard_units(any_numbers):
    "Convert any array of numbers to standard units."
    ...

In [124]:
usa_rt_su = ...

...
usa_rt_su

**Question 2.3** Plot the scatterplot in standard units below. Your plot should look the same except with different units. 

In [125]:
# Enter code here to visualize Temperature vs. Rainfall in standard units
usa_rt_su.scatter("temperature", "rainfall")

Why does it look the same? What is the significance of converting to standard units? What value do you think $r$ will take?
<!--
BEGIN QUESTION
name: q2_usa_rt_su
manual: true
points: 3
-->
<!-- EXPORT TO PDF -->

<hr style="color:Maroon;background-color:Maroon;border:0 none; height: 3px;">
Replace this text with your answer.
<hr style="color:Maroon;background-color:Maroon;border:0 none; height: 3px;">

**Question 2.4** Calculate the correlation coefficient, $r$.

*Hint:* Use the `usa_rt_su` table.  Section [15.2](https://www.inferentialthinking.com/chapters/15/1/Correlation.html) explains how to do this.

<!--
BEGIN QUESTION
name: q2_r
manual: true
points: 3
-->
<!-- EXPORT TO PDF -->

In [126]:
usa_r = ...
usa_r

In [ ]:
ok.grade("q2_r");

**Question 2.5** Find the regression line and create a function that uses the regression line to give a prediction on a given temperature in Celsius. In this case the regression line should take the form of:

$$\text{rainfall (MM) } = slope \times \text{temperature (Celsius)}.$$

*Hint:* Look at a previous lab, or 
[section 15.2](https://www.inferentialthinking.com/chapters/15/2/Regression_Line.html) if you need a reminder on finding the regression line.

In [128]:
def predict_rainfall(pred_temp):
    temperature_sd = ...
    rainfall_sd = ...
    slope = ...
    
    temperature_mean = ...
    rainfall_mean = ...
    intercept = ...
    
    prediction = ...
    return prediction

Below we plot the regression line using the prediction function we wrote!

In [129]:
def plot_data_and_line(dataset, x, y, point_0, point_1):
    """Makes a scatter plot of the dataset, along with a line passing through two points."""
    dataset.scatter(x, y, label="data")
    xs, ys = zip(point_0, point_1)
    plots.plot(xs, ys, label="regression line")
    plots.legend(bbox_to_anchor=(1.5,.8))

In [130]:
temp_min = np.min(usa_rain_and_temp.column("Temperature - (Celsius)"))
temp_max = np.max(usa_rain_and_temp.column("Temperature - (Celsius)"))
p_0 = [temp_min, predict_rainfall(temp_min)]
p_1 = [temp_max, predict_rainfall(temp_max)]
plot_data_and_line(usa_rain_and_temp, "Temperature - (Celsius)", "Rainfall - (MM)",p_0, p_1)

**Question 2.6** Using your prediction function, what would you predict the rainfall to be at $30^{\circ}$ celsius?
<!--
BEGIN QUESTION
name: q2_pred_30
manual: true
points: 3
-->
<!-- EXPORT TO PDF -->

In [131]:
pred_30 = ...
pred_30

In [ ]:
ok.grade("q2_pred_30");

### Bonus: Do your own Analysis!
Keep in mind that this prediction has been made using data from the United States! Try out your own regression analysis on another country in a similar fashion. More data can be downloaded on the [climate knowledge portal](https://climateknowledgeportal.worldbank.org/download-data)! Based on your results, how do you feel that the rainfall and temperature are related for that country? How do you think that relationship holds when looking at the world as a whole?

___


To submit:

1. Select `Run All` from the `Cell` menu to ensure that you have executed all cells, including the test cells.
2. Save and Checkpoint from the `File` menu.
3. Read through the notebook to make sure everything is fine.
4. Submit using the cell below.


# Submit
Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output.
**Please save before submitting!**

<!-- EXPECT 8 EXPORTED QUESTIONS -->

In [ ]:
# Save your notebook first, then run this cell to submit.
import jassign.to_pdf
jassign.to_pdf.generate_pdf('project2.ipynb', 'project2.pdf')
ok.submit()